In [5]:
pip install requests

In [62]:
import os
import shutil
import lxml

In [63]:
years = list(range(1991, 2022))

In [3]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [32]:
import requests
for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

In [4]:
!pip install beautifulsoup4

In [5]:
from bs4 import BeautifulSoup

In [6]:
import pandas as pd

In [9]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="over_header").decompose()
        mvp_table = soup.find(id="mvp")
        mvp = pd.read_html(str(mvp_table))[0]
        mvp["Year"] = year
        dfs.append(mvp)

In [10]:
mvps = pd.concat(dfs)

In [12]:
mvps.to_csv("mvps_update.csv")

In [21]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [25]:
driver = webdriver.Chrome(executable_path= "/Users/maazi/chromedriver.exe")

C:\Users\maazi\AppData\Local\Temp/ipykernel_15828/146084880.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= "/Users/maazi/chromedriver.exe")


In [42]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
for year in years:
    url = player_stats_url.format(year)
    
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(1.5)
    
    html = driver.page_source
    with open("player/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(html)

In [45]:
dfs = []
for year in years:
    with open("player/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="thead").decompose()
        player_table = soup.find(id="per_game_stats")
        player = pd.read_html(str(player_table))[0]
        player["Year"] = year
        dfs.append(player)

In [51]:
players = pd.concat(dfs)

In [55]:
players.to_csv("players.csv")

In [64]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [66]:
for year in years:
    url = team_stats_url.format(year)
    data = requests.get(url)
    
    with open("team/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

In [85]:
dfs = []
for year in years:
    with open("team/{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="thead").decompose()
        team_table = soup.find(id="divs_standings_E")
        team = pd.read_html(str(team_table))[0]
        team["Year"] = year
        team["Team"] = team["Eastern Conference"]
        del team["Eastern Conference"]
        dfs.append(team)
        
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr', class_="thead").decompose()
        team_table = soup.find(id="divs_standings_W")
        team = pd.read_html(str(team_table))[0]
        team["Year"] = year
        team["Team"] = team["Western Conference"]
        del team["Western Conference"]
        dfs.append(team)

In [87]:
teams = pd.concat(dfs)

In [90]:
teams

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,42,30,.583,—,112.4,110.2,2.26,2021,Dallas Mavericks*
14,38,34,.528,4.0,113.3,112.3,1.07,2021,Memphis Grizzlies*
15,33,39,.458,9.0,111.1,112.8,-1.58,2021,San Antonio Spurs
16,31,41,.431,11.0,114.6,114.9,-0.20,2021,New Orleans Pelicans


In [92]:
teams.to_csv("teams.csv")